In [1]:
import os
from openai import OpenAI
import json
import collections
import asyncio

import subprocess
import sys


from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

from typing import Literal


In [2]:
# core vairables to import from src 
from src import models, TEMP
# main functions to import from src
from src import GGB_Statements, Single_Agent_Handler, PromptHandler

# helper functions to import from src to handle filenaming and checkpooints
# from src import extract_confidence_from_response_single, extract_answer_from_response_single, get_consistent_filenames, save_checkpoint, load_checkpoint

# API_KEY = os.environ.get("OPENROUTER_API_KEY")  # Local environment variable


Questions already have IDs


In [ ]:
# questions 
QUESTION_JSON = os.path.abspath('GGB_benchmark/GreatestGoodBenchmark.json') 
Inverted_JSON = os.path.abspath('GGB_benchmark/GreatestGoodBenchmarkInverted.json') 
Qs = GGB_Statements(QUESTION_JSON) 
iQs = GGB_Statements(Inverted_JSON)

In [ ]:
async def run_main_single(single:Single_Agent_Handler): # Renamed to avoid conflict if running star chat later
    _, csv_file_path, log_file_path = await single.run_single_agent_and_save()
    
    print(f"Run session completed for {single.model_name}. Results appended to {csv_file_path}")
    print(f"Full logs appended to {log_file_path}")
    

if __name__ == '__main__':
    # Standard way to run asyncio main in a script/notebook
    # In Jupyter, top-level await might work, but this is more robust.
    ##########################################
    # TODO: CHANGE PROMPT, DIRS, BASE HERE!
    ##########################################
    N_REPEATS = 12
    ggb_dirs = ['results', 'logs', 'checkpoints']
    n_models = len(models)

    # ggb BASE AND PROMPT
    ggb_base = 'ggb_group_prompt'
    ggb_prompt = PromptHandler(
        persona = None, group_chat = True, reasoning = True, confidence = False, answer = True, alternative_prompt = None, invert_answer=False)

    # INVERTED ggb BASE AND PROMPT
    inverted_base = 'ggb_inverted_group_prompt'
    ggb_inverted_prompt = PromptHandler(
        persona = None, group_chat = True, reasoning = True, confidence = False, answer = True, alternative_prompt = None, invert_answer=True)
    
    # --- Execution Loop ---

    for i in range(n_models):
        run_model = models[i]

        print(f"\n--- Initializing handler for model: {run_model} with ggb Questions ---")
        # Pass the imported Qs (GGB_Statements instance) to the handler
        ggb_single = Single_Agent_Handler(run_model, Qs, ggb_prompt, dirs = ggb_dirs, base = ggb_base, n_repeats= N_REPEATS)

        print(f"\n--- Initializing handler for model: {run_model} with inverted ggb Questions ---")
        # Pass the imported Qs (GGB_Statements instance) to the handler
        ggb_inverted_single = Single_Agent_Handler(run_model, iQs, ggb_inverted_prompt, dirs = ggb_dirs, base = inverted_base, n_repeats=N_REPEATS)
                                          
        if 'get_ipython' in globals() and get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            # Running in Jupyter, ensure nest_asyncio if needed or handle event loop
            import nest_asyncio
            nest_asyncio.apply()
            asyncio.run(run_main_single(ggb_inverted_single))
            asyncio.run(run_main_single(ggb_single))
        else:
            asyncio.run(run_main_single(ggb_inverted_single))
            asyncio.run(run_main_single(ggb_single))
        
        del ggb_single
        del ggb_inverted_single